In [ ]:
# BeautifulSoup을 사용하지않고 selenium만을 활용해서 데이터 수집을 진행 하였다.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import csv
# -----------------------------------------------------------------------------------------------------------------
# 브라우저 진입
browser = webdriver.Chrome()
browser.get('https://new.land.naver.com/complexes?ms=37.3595704,127.105399,16&a=APT:ABYG:JGC:PRE&e=RETAIL')
# 네이버 부동산 사이트를 참고한 데이터 
time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
#링크를 크롤링 하기 위한 사이트 링크 파도타기
element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
# 지도에 지역부분을 클릭해준다.
time.sleep(1)

element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
# 지역부분을 클릭해주고나서, 서울시를 클릭하여 서울시의 범위만을 지정한다.
time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
# 서울시의 모든 '구' 를 리스트화 시켜서 append 함
sigu_elements = browser.find_elements(By.CLASS_NAME, 'area_list--district')
sigu_list = []
  
for sigu_search in sigu_elements:
    # \n을 기준으로 문장을 분리하고 리스트에 추가
    sentences = sigu_search.text.split('\n')
    sigu_list.extend(sentences)
# 본래의 데이터가 강남구\n강서구\n강북구\n 처럼 되어있어서 \n을 기준점으로 잡아 스플릿을 돌려서
# 리스트에 하나 하나의 값으로 추가시켜주는 과정을 진행 하고 있다.
# -----------------------------------------------------------------------------------------------------------------
# 데이터 값 추출을 위한 for문
# 사용될 리스트를 코드가 진행되기 전에 선언한다.
apt_list = []
dong_list = []
price_list = []
final_list = []
# -----------------------------------------------------------------------------------------------------------------
# 위에 과정에서 ['강남구','강북구','강서구' ....] 리스트를 만들었으니, 그 길이만큼 for문을 돌려서
# XAPTH를 활용해서 for문이 진행될 때, 해당 인덱스 값의 XPATH를 불러올 수 있도록 코드를 만들었다.
# 0번 XAPTH는 존재하지 않기 때문에, 숫자가 들어올 Sigu_data+1 을 넣어서 오류가 나오지않게 진행한다.
for sigu_data in range(len(sigu_list)):
    sigu_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
    time.sleep(1)
    dong_elements = browser.find_elements(By.CLASS_NAME, 'area_list--district')
# '구'리스트를 뽑아왔으니, 이제 '동' 리스트를 만들어 처리방식은 아까와 동일하게 처리한다.
    for dong_search in dong_elements:
        # \n을 기준으로 문장을 분리하고 리스트에 추가
        sentences = dong_search.text.split('\n')
        dong_list.extend(sentences)
# -----------------------------------------------------------------------------------------------------------------
# 중첩 for문을 사용해서 '강남구'를 클릭하고 '개포통', '논현동', '대치동' ... 순서대로  위와 동일하게
# XPATH를 활용해서 맞는 링크를 클릭할 수 있도록 진행한다.
    for dong_data in range(len(dong_list)):
        dong_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{dong_data+1}]/label').click()
        time.sleep(1)
        apt_elements = browser.find_elements(By.CLASS_NAME, 'complex_title')
        apt_list_recycle = []
# 이번엔 해당 '동' 에 존재하는 아파트의 리스트를 위와 동일한 방식으로 처리하되, 이 데이터는 \n 이 없어서 간단한 방식으로
# 처리하였다. 추후에 모든 아파트의 데이터가 필요할지도 몰라서 for문이 돌때마다 초기화를 시켜줄 리스트와 모든 아파트 데이터를
# 포함하고 있는 리스트까지 총 두개의 리스트에게 append 시켜주고 있다.
        for apt_search in apt_elements:
            sentences = apt_search.text
            if sentences:
                apt_list.append(sentences)
                apt_list_recycle.append(sentences)
# -----------------------------------------------------------------------------------------------------------------
# 위에 '구','동' 을 돌리던 for문과 동일하게 아파트 리스트의 for문을 돌려서 해당 링크를 눌러서 작업을 반복하게 해준다.
# 'execute_script' 코드를 활용해서, 아파트 리스트가 많아, 스크롤 다운을 진행해야 하는데 'arguments[0].click()' 을
# 활용해서 스크롤 다운 대신에 그 값을 찾아서 오류가 나지 않게끔 코드를 진행시켜 준다.
        for apt_data in range (len(apt_list_recycle)):
            button = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[3]/ul/li[{apt_data+1}]/a')
            browser.execute_script('arguments[0].click()', button)
            time.sleep(1)
# 아파트의 매물이 없는경우에는 (By.LINK_TEXT, '시세/실거래가') 값이 존재하지 않는 경우가 있어서, try를 활용하여,
# 해당 링크가 있다면 눌러서 진행하고, 해당 링크가 존재하지 않는다면 except 처리하여, 해당 데이터의 값은
# '평수' ,'가격' 에 'None' 값을 대신 넣어서 해결해줄 생각이다.
            try:
                apt_won = browser.find_element(By.LINK_TEXT, '시세/실거래가')
                apt_won.click()
                time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
# 이제 (By.LINK_TEXT, '시세/실거래가') 링크를 클릭해서 들어오면, 해당 아파트의 평수의 갯수들이 나오게 되는데,
# 이것도 마찬가지로 평수가 적을 경우는 더보기 링크가 없기 때문에, 위와 동일하게 try 를 활용해서 처리하였다.
                try:
                    arrow = browser.find_element(By.XPATH, '//*[@id="detailContents2"]/div[1]/div/div[2]/button')
                    if arrow.text == '더보기':
                        arrow.click()
                        time.sleep(1)
                        area_elements = browser.find_elements(By.CLASS_NAME, 'detail_sorting_tab')
                        time.sleep(1)
# 평수만큼 for문을 또 돌려야 하기 떄문에, '아파트' 리스트와 동일하게 처리하였다.
                        area_list = []
                        for area_search in area_elements:
                            sentences = area_search.text
                            if sentences:
                                area_list.append(sentences)
# 존재하는 평수의 갯수만큼 for을 돌리는데, 해당 데이터에는 '전세','월세','매매' 데이터값을 포함하고 있어서
# 뒤에꺼 3개는 빼고 for문을 돌려주고 있다. 이렇게해야 오류가 나지 않는다.
                        for area_data in area_list[:-3]:
                            browser.find_element(By.LINK_TEXT, f'{area_data}').click()
                            time.sleep(1)
# ----------------------------------------------------------------------------------------------------------------- 
                            # 해당 평수의 '상한가'를 지정해서 text화 시키고, 가격이 존재하면 try를 활용해서 final_list에
                            # '구','동','아파트','평수','가격' 순서대로 최종리스트에 값을 넣어주고 있다.
                            try:
                                price = browser.find_element(By.XPATH, '//*[@id="tabpanel1"]/div[3]/div[1]/div/div[2]/strong').text
                                price_list.append(price)
                                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,price))
                                time.sleep(1)
                            # '상한가' 가격이 존재하지 않다면, '평수' 까지만 리스트에 저장하고, 뒤에값은 'None'을 삽입 하였다.
                            except:
                                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,'None'))
                            # 더보기 링크를 다시한번 클릭해서 더보기 창을 닫아준다.
                        if arrow.text == '더보기':
                            arrow.click()
                            time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
                # 위에 코드와 동일한 코드인데, 처음에 더보기 링크가 존재하지 않을 때에, 예외처리 되서 넘어온 경우
                # 어쨋든, '평수' 가 존재는 하지만, '평수' 의 갯수가 적어서 더보기 링크가 없는 경우이기 때문에,
                # 위와 동일한 코드 흐름 방식을 적용시켜서 누락되는 데이터가 없게 하였다.
                except:
                    area_elements = browser.find_elements(By.CLASS_NAME, 'detail_sorting_tab')
                    time.sleep(1)
                    area_list = []
                    for area_search in area_elements:
                        sentences = area_search.text
                        if sentences:
                            area_list.append(sentences)
                    for area_data in area_list[:-3]:
                        browser.find_element(By.LINK_TEXT, f'{area_data}').click()
                        time.sleep(1)
                        try:                                         
                            price = browser.find_element(By.XPATH, '//*[@id="tabpanel1"]/div[3]/div[1]/div/div[2]/strong').text
                            price_list.append(price)
                            final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,price))
                            time.sleep(1)
                        except:
                            final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,'None'))
# -----------------------------------------------------------------------------------------------------------------
            # 맨처음에 (By.LINK_TEXT, '시세/실거래가') 링크 텍스트가 존재하지 않을 때, 예외처리를 했던 코드인데,
            # '평수' 와 '가격' 쪽에 'None' 값을 넣어서, 마지막 처리를 해주고 있다. 그리고 모든 코드의 흐름이 끝나면
            # 창닫기 버튼을 눌러서 해당 아파트의 창을 닫아주는 작업을 진행하고 있다.
            except:
                price_list.append('None')
                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],'None','None'))
            # 아파트가 아예 존재하지 않는 '구' 가 존재하는 경우가 있는데, 이 경우에 예외처리를 진행하기 위해서 닫기 버튼이
            # 없을 때 예외처리를 할 수 있는 코드를 작성하게 되었다.
            try:
                close_button = browser.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/button').click()
            except:
                element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
# -----------------------------------------------------------------------------------------------------------------
            # 해당 '동' 하나의 아파트 데이터를 크롤링 하였다면, 다시 재귀해서 돌아가기 위해 지역부분을 다시 클릭해준다.
            # 서울시를 누른후에 다시 해당 '구' 를 클릭후 다시 똑같은 '동' 으로 돌아가도록 클릭하게 하여,
            # 다음 아파트를 크롤링 하게끔 코드를 진행하게 하였다.
            element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
            time.sleep(1)
    
            element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
            time.sleep(1)
            
            browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
            time.sleep(1)
                    
            dong_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{dong_data+1}]/label').click()
            time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
        # 해당 '동'의 모든 아파트의 크롤링을 마치고 나면, 위와 동일하게 지역부분을 다시 클릭해주고, 서울시를 누른다음에,
        # 다시 해당 '구' 로 돌아오게하여, 다음 '동' 의 아파트를 크롤링 하게 만들어 준다.
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
        time.sleep(1)
            
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
        time.sleep(1)
        
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
        time.sleep(1)
        
        browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
        time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
    # 해당하는 '구' 에 '동' 에 있는 모든 아파트를 크롤링 하였다면, 다음 '구' 로 넘어가기 위해서, 지역부분을 클릭한후에
    # '서울시' 까지만 클릭해주게 해주고, 쭉쭉 for문으로 재귀하게 만들고 있다.
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
    time.sleep(1)
            
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
    time.sleep(1)
        
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
    time.sleep(1)
# ----------------------------------------------------------------------------------------------------------------- 
# 최종적으로 ['구','동','아파트','평수','가격'] 를 맨 상단에 삽입하고, final_list를 csv 파일로 쓰기를 진행해서 모든 데이터를 삽입한다. 
def apt_xml(file):
    with open(file, "w", newline='', encoding='UTF-8') as file_w:
        writer = csv.writer(file_w)
        writer.writerow(sort_list)
        writer.writerows(final_list)
sort_list = ['구','동','아파트','평수','가격']
apt_xml("apt_up.csv")
print('apt_up 저장 완료!')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import csv
# -----------------------------------------------------------------------------------------------------------------
browser = webdriver.Chrome()
browser.get('https://new.land.naver.com/complexes?ms=37.3595704,127.105399,16&a=APT:ABYG:JGC:PRE&e=RETAIL')
time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
time.sleep(1)

element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
sigu_elements = browser.find_elements(By.CLASS_NAME, 'area_list--district')
sigu_list = []
  
for sigu_search in sigu_elements:
    sentences = sigu_search.text.split('\n')
    sigu_list.extend(sentences)
# -----------------------------------------------------------------------------------------------------------------
apt_list = []
dong_list = []
price_list = []
final_list = []
# -----------------------------------------------------------------------------------------------------------------
for sigu_data in range(len(sigu_list)):
    sigu_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
    time.sleep(1)
    dong_elements = browser.find_elements(By.CLASS_NAME, 'area_list--district')
    for dong_search in dong_elements:
        sentences = dong_search.text.split('\n')
        dong_list.extend(sentences)
# -----------------------------------------------------------------------------------------------------------------
    for dong_data in range(len(dong_list)):
        dong_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{dong_data+1}]/label').click()
        time.sleep(1)
        apt_elements = browser.find_elements(By.CLASS_NAME, 'complex_title')
        apt_list_recycle = []
        for apt_search in apt_elements:
            sentences = apt_search.text
            if sentences:
                apt_list.append(sentences)
                apt_list_recycle.append(sentences)
# -----------------------------------------------------------------------------------------------------------------
        for apt_data in range (len(apt_list_recycle)-1):
            button = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[3]/ul/li[{apt_data+1}]/a')
            browser.execute_script('arguments[0].click()', button)
            time.sleep(1)
            try:
                apt_won = browser.find_element(By.LINK_TEXT, '시세/실거래가')
                apt_won.click()
                time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
                try:
                    arrow = browser.find_element(By.XPATH, '//*[@id="detailContents2"]/div[1]/div/div[2]/button')
                    if arrow.text == '더보기':
                        arrow.click()
                        time.sleep(1)
                        area_elements = browser.find_elements(By.CLASS_NAME, 'detail_sorting_tab')
                        time.sleep(1)
                        area_list = []
                        for area_search in area_elements:
                            sentences = area_search.text
                            if sentences:
                                area_list.append(sentences)
                        for area_data in area_list[:-3]:
                            browser.find_element(By.LINK_TEXT, f'{area_data}').click()
                            time.sleep(1)
# ----------------------------------------------------------------------------------------------------------------- 
                            try:
                                price = browser.find_element(By.XPATH, '//*[@id="tabpanel1"]/div[3]/div[1]/div/div[2]/strong').text
                                price_list.append(price)
                                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,price))
                                time.sleep(1)
                            except:
                                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,'None'))
                        if arrow.text == '더보기':
                            arrow.click()
                            time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
                except:
                    area_elements = browser.find_elements(By.CLASS_NAME, 'detail_sorting_tab')
                    time.sleep(1)
                    area_list = []
                    for area_search in area_elements:
                        sentences = area_search.text
                        if sentences:
                            area_list.append(sentences)
                    for area_data in area_list[:-3]:
                        browser.find_element(By.LINK_TEXT, f'{area_data}').click()
                        time.sleep(1)
                        try:                                         
                            price = browser.find_element(By.XPATH, '//*[@id="tabpanel1"]/div[3]/div[1]/div/div[2]/strong').text
                            price_list.append(price)
                            final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,price))
                            time.sleep(1)
                        except:
                            final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],area_data,'None'))
# -----------------------------------------------------------------------------------------------------------------
            except:
                price_list.append('None')
                final_list.append((sigu_list[sigu_data],dong_list[dong_data],apt_list_recycle[apt_data],'None','None'))
            try:
                close_button = browser.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/button').click()
            except:
                element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
# -----------------------------------------------------------------------------------------------------------------
            element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
            time.sleep(1)
    
            element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
            time.sleep(1)
            
            browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
            time.sleep(1)
                    
            dong_element = browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{dong_data+1}]/label').click()
            time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
        time.sleep(1)
            
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
        time.sleep(1)
        
        element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
        time.sleep(1)
        
        browser.find_element(By.XPATH, f'//*[@id="region_filter"]/div/div/div[2]/ul/li[{sigu_data+1}]/label').click()
        time.sleep(1)
# -----------------------------------------------------------------------------------------------------------------
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
    time.sleep(1)
            
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/a/span[1]').click()
    time.sleep(1)
        
    element = browser.find_element(By.XPATH, '//*[@id="region_filter"]/div/div/div[2]/ul/li[1]/label').click()
    time.sleep(1)
# ----------------------------------------------------------------------------------------------------------------- 
def apt_xml(file):
    with open(file, "w", newline='', encoding='UTF-8') as file_w:
        writer = csv.writer(file_w)
        writer.writerow(sort_list)
        writer.writerows(final_list)
sort_list = ['구','동','아파트','평수','가격']
apt_xml("apt_test")
print('apt_test 저장 완료!')

In [ ]:
def apt_xml(file):
    with open(file, "w", newline='', encoding='UTF-8') as file_w:
        writer = csv.writer(file_w)
        writer.writerow(sort_list)
        writer.writerows(final_list)
sort_list = ['구','동','아파트','평수','가격']
apt_xml("apt_test")
print('apt_test 저장 완료!')